In [43]:
# Carga de librerias
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go # para gráficos
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import funciones as fn

# Forma
from IPython.display import display, Markdown
palette_color=['#d4afb9', '#d1cfe2', '#9cadce', '#7ec4cf', '#52b2cf']
palette_color2 = ['#d4afb9', '#d1cfe2', '#9cadce', '#7ec4cf', '#52b2cf', '#34a0a4', '#80ced7', '#b5e48c', '#cdb4db', '#ffafcc']


In [44]:
# Conectarse a BD 
conn = sql.connect('Data/db_movies.db')
cur = conn.cursor()

In [45]:
# Para ver las tablas que hay en la base de datos
cur.execute("select name from sqlite_master where type='table' ")
cur.fetchall()

[('ratings',), ('movies',)]

In [46]:
# traer tabla de BD a python 
movies = pd.read_sql("select *  from movies", conn)
ratings = pd.read_sql('select * from ratings', conn)

In [47]:
# prompt: Genera un código que muestre esas tablas de datos

print("'movies' Table:")
display(movies.head())  

print("'ratings' Table:")
display(ratings.head())  


'movies' Table:


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


'ratings' Table:


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [48]:
def check_df(dataframe):
    # Dimensiones base general
    display(Markdown('**Dimensiones base general**'))
    display(dataframe.shape)

    # Dimensiones sin duplicados
    display(Markdown('**Dimensiones sin duplicados**'))
    display(dataframe.drop_duplicates().shape)

    # Tipos de datos
    display(Markdown('**Tipos**'))
    display(dataframe.dtypes)

    # Valores nulos
    display(Markdown('**Nulos**'))
    display(dataframe.isnull().sum())

print("'movies:")
check_df(movies)
print("\n'ratings:")
check_df(ratings)

'movies:


**Dimensiones base general**

(9742, 3)

**Dimensiones sin duplicados**

(9742, 3)

...

(100836, 4)

**Tipos**

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

**Nulos**

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [49]:
# Distribución de las calificaciones

# muestra cuántas veces aparece cada calificación, ordenadas de la más frecuente a la menos frecuente.
cr = pd.read_sql(""" select rating,
                    count(*) as conteo
                    from ratings
                    group by rating
                    order by conteo desc""", conn)
cr

,rating,conteo
0,4.0,26818
1,3.0,20047
2,5.0,13211
3,3.5,13136
4,4.5,8551
5,2.0,7551
6,2.5,5550
7,1.0,2811
8,1.5,1791
9,0.5,1370


In [50]:
data = go.Bar(x=cr.rating, marker_color=palette_color2, y=cr.conteo, text=cr.conteo, textposition="outside")
Layout = go.Layout(title="Count of ratings", xaxis={'title':'Rating'}, yaxis={'title':'Count'})
go.Figure(data,Layout)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': [#d4afb9, #d1cfe2, #9cadce, #7ec4cf, #52b2cf,
                                   #34a0a4, #80ced7, #b5e48c, #cdb4db, #ffafcc]},
              'text': array([26818., 20047., 13211., 13136.,  8551.,  7551.,  5550.,  2811.,  1791.,
                              1370.]),
              'textposition': 'outside',
              'type': 'bar',
              'x': array([4. , 3. , 5. , 3.5, 4.5, 2. , 2.5, 1. , 1.5, 0.5]),
              'y': array([26818, 20047, 13211, 13136,  8551,  7551,  5550,  2811,  1791,  1370])}],
    'layout': {'template': '...',
               'title': {'text': 'Count of ratings'},
               'xaxis': {'title': {'text': 'Rating'}},
               'yaxis': {'title': {'text': 'Count'}}}
})

In [51]:
# Cacular cuántas películas calificó cada usuario
rating_users = pd.read_sql(''' select userId,
                                count(*) as cnt_rat
                                from ratings
                                group by userId
                                order by cnt_rat asc
                                ''',conn )
rating_users

,userId,cnt_rat
0,53,20
1,147,20
2,189,20
3,194,20
4,207,20
...,...,...
605,274,1346
606,448,1864
607,474,2108
608,599,2478


In [52]:
#plt.hist(rating_users['cnt_rat'], bins=10, color=palette_color2[0])
#plt.title('Hist frecuencia de número de calificaciones por usuario')
#plt.xlabel('Número de calificaciones')
#plt.ylabel('Frecuencia')

fig = px.histogram(rating_users, x='cnt_rat', color_discrete_sequence=palette_color, title='Histograma frecuencia de número de calificaciones por usuario')
fig.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

**Distribución sesgada:** La mayoría de los usuarios han realizado muy pocas calificaciones. La barra más alta está cerca del cero, lo que indica que una gran cantidad de usuarios solo han hecho un número pequeño de calificaciones (probablemente menos de 100).

**Usuarios con más calificaciones:** Aunque la mayoría ha hecho pocas calificaciones, algunos usuarios han hecho un número considerablemente mayor, como se observa en las barras más pequeñas que se extienden hasta 2500 calificaciones.

**Cola larga:** Hay una cola larga a la derecha del gráfico. Esto significa que un pequeño número de usuarios ha realizado una cantidad significativamente mayor de calificaciones en comparación con el resto.

**Interpretación general:** La mayoría de los usuarios son inactivos o califican pocos ítems, mientras que un grupo reducido de usuarios contribuye mucho más a las calificaciones totales. Es una distribución típica de plataformas donde una minoría activa es responsable de una gran cantidad de interacciones.

In [72]:
# Descripción ratings
rating_users.describe()

,userId,cnt_rat
count,610.000000,610.000000
mean,305.500000,165.304918
std,176.236111,269.480584
min,1.000000,20.000000
25%,153.250000,35.000000
50%,305.500000,70.500000
75%,457.750000,168.000000
max,610.000000,2698.000000


Como en el histograma se obseva que la mayoria de calificaciones estan entre 0 y 500 se

In [73]:
## excluir usuarios con menos de 500 películas calificadas
rating_users2 = pd.read_sql('''select userId,
                                count(*) as cnt_rat
                                from ratings
                                group by userId
                                having cnt_rat <=500
                                order by cnt_rat asc
                                ''',conn )
rating_users2

,userId,cnt_rat
0,53,20
1,147,20
2,189,20
3,194,20
4,207,20
...,...,...
562,509,467
563,368,469
564,381,474
565,57,476


In [74]:
rating_users2.describe()

,userId,cnt_rat
count,567.000000,567.000000
mean,303.340388,107.199295
std,174.739864,105.154761
min,1.000000,20.000000
25%,152.500000,34.000000
50%,304.000000,63.000000
75%,453.500000,139.500000
max,609.000000,500.000000


In [75]:
fig = px.histogram(rating_users2, color_discrete_sequence=palette_color, x='cnt_rat', title='Histograma frecuencia de número de calificaciones por usuario')
fig.show()

In [76]:
# calificación de cada película
rating_movie = pd.read_sql('''select movieId,
                                count(*) as cnt_rat
                                from ratings
                                group by movieId
                                order by cnt_rat desc
                                ''',conn )
rating_movie

,movieId,cnt_rat
0,356,329
1,318,317
2,296,307
3,593,279
4,2571,278
...,...,...
9719,96,1
9720,83,1
9721,77,1
9722,55,1


In [77]:
fig = px.histogram(rating_movie, color_discrete_sequence=palette_color, x='cnt_rat', title='Histograma frecuencia de número de calificaciones por película')
fig.show()

In [78]:
rating_movie.describe()

,movieId,cnt_rat
count,9724.000000,9724.000000
mean,42245.024373,10.369807
std,52191.137320,22.401005
min,1.000000,1.000000
25%,3245.500000,1.000000
50%,7300.000000,3.000000
75%,76739.250000,9.000000
max,193609.000000,329.000000


In [79]:
# Películas que tengan más de 10 calificaciones
rating_movie2 = pd.read_sql(''' select movieId,
                                count(*) as cnt_rat
                                from ratings
                                group by movieId
                                having cnt_rat >= 10
                                order by cnt_rat desc
                                ''',conn )
rating_movie2

,movieId,cnt_rat
0,356,329
1,318,317
2,296,307
3,593,279
4,2571,278
...,...,...
2264,174,10
2265,169,10
2266,97,10
2267,94,10


In [80]:
fig = px.histogram(rating_movie2, color_discrete_sequence=palette_color, x='cnt_rat', title= 'Histograma frecuencia de número de calificaciones por pelicula')
fig.show()

In [81]:
rating_movie2.describe()

,movieId,cnt_rat
count,2269.000000,2269.000000
mean,20530.586161,35.749669
std,35185.840333,35.986989
min,1.000000,10.000000
25%,1345.000000,14.000000
50%,3256.000000,22.000000
75%,8958.000000,43.000000
max,187593.000000,329.000000


Lectura de sql

In [53]:
# Conectarse a BD 
conn = sql.connect('Data/movies2.db')
cur = conn.cursor()

In [54]:
fn.ejecutar_sql('preprocesamiento_n.sql', cur)

In [55]:
cur.execute("select name from sqlite_master where type='table' ")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',)]

In [56]:
# consultar trayendo para pandas
df_final = pd.read_sql("select * from full_ratings", conn)
df_final

,user_id,movie_id,rating,timestamp,movie_title,movie_genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
45420,609,589,3.0,847220990,Terminator 2: Judgment Day (1991),Action|Sci-Fi
45421,609,590,4.0,847220802,Dances with Wolves (1990),Adventure|Drama|Western
45422,609,592,3.0,847220802,Batman (1989),Action|Crime|Thriller
45423,609,786,3.0,847221025,Eraser (1996),Action|Drama|Thriller


In [57]:
# Separar el título y el año
df_final['movie_year'] = df_final['movie_title'].str.extract(r'\((\d{4})\)')  # Extraer el año
df_final['movie_clean_title'] = df_final['movie_title'].str.replace(r'\s*\(\d{4}\)', '', regex=True).str.strip()  # Extraer el título limpio

In [58]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45425 entries, 0 to 45424
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            45425 non-null  int64  
 1   movie_id           45425 non-null  int64  
 2   rating             45425 non-null  float64
 3   timestamp          45425 non-null  int64  
 4   movie_title        45425 non-null  object 
 5   movie_genres       45425 non-null  object 
 6   movie_year         45425 non-null  object 
 7   movie_clean_title  45425 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 2.8+ MB


In [59]:
# Filtrar las filas donde la columna "movie_year" tenga valores nulos
nulos = df_final[df_final['movie_year'].isnull()]
nulos

,user_id,movie_id,rating,timestamp,movie_title,movie_genres,movie_year,movie_clean_title


In [60]:
# Definir un diccionario de títulos de películas y sus años correspondientes
nuevos_años = {
    'Babylon 5': 1994,
    'Ready Player One': 2018,
    'Black Mirror': 2018,
    'Maria Bamford: Old Baby': 2017,
    'Generation Iron 2': 2017,
    'Nocturnal Animals': 2016,
    'Moonlight': 2016,
    'Paterson': 2016
}

# Iterar sobre el diccionario y actualizar los años correspondientes en el DataFrame
for titulo, año in nuevos_años.items():
    df_final.loc[df_final['movie_title'] == titulo, 'movie_year'] = año

In [61]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45425 entries, 0 to 45424
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            45425 non-null  int64  
 1   movie_id           45425 non-null  int64  
 2   rating             45425 non-null  float64
 3   timestamp          45425 non-null  int64  
 4   movie_title        45425 non-null  object 
 5   movie_genres       45425 non-null  object 
 6   movie_year         45425 non-null  object 
 7   movie_clean_title  45425 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 2.8+ MB


In [62]:
# Agrupar por el título de la película y contar el número de ocurrencias
movie_views = df_final.groupby('movie_title')['user_id'].count().sort_values(ascending=False)

# Crear un DataFrame con los datos de las 10 películas más vistas
top_movies = movie_views.head(10).reset_index(name='num_views')

# Graficar el número de vistas por película utilizando Plotly Express
fig = px.bar(top_movies, x='movie_title', y='num_views',
             title='Top 10 Películas Más Vistas',
             labels={'movie_title': 'Película', 'num_views': 'Número de Veces Vista'},
             color_discrete_sequence=palette_color)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [95]:
# Agrupar por la fecha nueva y contar el número de películas vistas
movies_by_date = df_final.groupby('fecha_nueva')['movie_title'].count().reset_index()

# Graficar el número de películas vistas por fecha utilizando Plotly Express
fig = px.line(movies_by_date, x='fecha_nueva', y='movie_title',
              title='Cantidad de Películas Vistas por Fecha',
              labels={'fecha_nueva': 'Fecha', 'movie_title': 'Número de Películas Vistas'},
              color_discrete_sequence=palette_color)
fig.show()

In [64]:
#exportar datos
# Crear una conexión a la base de datos en la carpeta 'salidas'
conn = sql.connect('salidas/mi_base_de_datos.db')

# Exportar el DataFrame 'df_final' a una tabla de SQLite llamada 'df_final'
df_final.to_sql('df_final', conn, index=False, if_exists='replace')

45425